In [1]:
 from pyspark.sql import SparkSession
 import getpass
 username = getpass.getuser()
 spark= SparkSession. \
 builder. \
 config('spark.ui.port','0'). \
 config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
 enableHiveSupport(). \
 master('yarn'). \
 getOrCreate()

# CONTENTS COVERED
    # Enforcing schema
    # Different types of creating df (df from test data,reader api, .sql...ect)
    # Handling date fields
    # Expression handling
    # Drop columns & drop dup row

In [2]:
# 1. Write PySpark code to create a new dataframe with the data given below
#  having 2 columns (‘season’) and (‘windspeed’).
#  # Data
#  [("Spring", 12.3),
#  ("Summer", 10.5),
#  ("Autumn", 8.2),
#  ("Winter", 15.1)]

In [3]:
tst_data = [("Spring", 12.3), ("Summer", 10.5),("Autumn", 8.2),("Winter", 15.1)]
sctructure = ('season string,windspeed float')
season_df = spark.createDataFrame(tst_data,sctructure)
season_df.show()

+------+---------+
|season|windspeed|
+------+---------+
|Spring|     12.3|
|Summer|     10.5|
|Autumn|      8.2|
|Winter|     15.1|
+------+---------+



In [4]:
season_df.printSchema()

root
 |-- season: string (nullable = true)
 |-- windspeed: float (nullable = true)



In [5]:
# If schema is infered , will not have col names
season_df_tst = spark.createDataFrame(tst_data)
season_df_tst.show()

+------+----+
|    _1|  _2|
+------+----+
|Spring|12.3|
|Summer|10.5|
|Autumn| 8.2|
|Winter|15.1|
+------+----+



In [6]:
# 2)  load  data into a Dataframe and enforce schema using StructType from  "/public/trendytech/datasets/library_data.json"

In [7]:
! hadoop fs -head /public/trendytech/datasets/library_data.json

{"library_name": "Central Library","location": "City Center","books": [{"book_id": "B001","book_name": "The Great Gatsby","author": "F. Scott Fitzgerald","copies_available": 5},{"book_id": "B002","book_name": "To Kill a Mockingbird","author": "Harper Lee","copies_available": 3}],"members": [{"member_id": "M001","member_name": "John Smith","age": 28,"books_borrowed": ["B001"]},{"member_id": "M002","member_name": "Emma Johnson","age": 35,"books_borrowed": []}]},
{"library_name": "Community Library","location": "Suburb","books": [{"book_id": "B003","book_name": "1984","author": "George Orwell","copies_available": 2},{"book_id": "B004","book_name": "Pride and Prejudice","author": "Jane Austen","copies_available": 4}],"members": [{"member_id": "M003","member_name": "Michael Brown","age": 42,"books_borrowed": ["B003","B004"]},{"member_id": "M004","member_name": "Sophia Davis","age": 31,"books_borrowed": ["B004"]}]}


In [8]:
# # NESTED SCHEMA
# {"library_name": "Central Library","location": "City Center",
 
#  "books": [{"book_id": "B001","book_name": "The Great Gatsby","author": "F. Scott Fitzgerald","copies_available": 5},
#           {"book_id": "B002","book_name": "To Kill a Mockingbird","author": "Harper Lee","copies_available": 3}],
 
#  "members": [{"member_id": "M001","member_name": "John Smith","age": 28,"books_borrowed": ["B001"]},
#              {"member_id": "M002","member_name": "Emma Johnson","age": 35,"books_borrowed": []}]
# },

In [9]:
from pyspark.sql.types import *

In [10]:
book_schema = StructType([
StructField("library_name",StringType()),
StructField("location",StringType()),
StructField("books",ArrayType(StructType([StructField("book_id",StringType()),StructField("book_name",StringType()),StructField("author",StringType()),StructField("copies_available",IntegerType())]))),
StructField("members",ArrayType(StructType([StructField("member_id",StringType()),StructField("member_name",StringType()),StructField("age",IntegerType()),StructField("books_borrowed",StringType())])))
	])	

In [11]:
book_df = spark.read \
.format('json') \
.schema(book_schema) \
.load('/public/trendytech/datasets/library_data.json')

In [12]:
# book_df.show()
book_df.printSchema()

root
 |-- library_name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- books: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- book_id: string (nullable = true)
 |    |    |-- book_name: string (nullable = true)
 |    |    |-- author: string (nullable = true)
 |    |    |-- copies_available: integer (nullable = true)
 |-- members: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- member_id: string (nullable = true)
 |    |    |-- member_name: string (nullable = true)
 |    |    |-- age: integer (nullable = true)
 |    |    |-- books_borrowed: string (nullable = true)



In [13]:
# 3. Given the dataset (/public/trendytech/datasets/train.csv),create a Dataframe using PySpark and perform the following operations

In [14]:
! hadoop fs -head /public/trendytech/datasets/train.csv

train_number,train_name,seats_available,passenger_name,age,ticket_number,seat_number
123,Express,100,John,25,T123,A1
123,Express,100,Emma,30,T124,B2
456,Superfast,150,Michael,35,T125,C3
456,Superfast,150,Sophia,40,T126,D4
789,Local,50,William,28,T127,E5
789,Local,50,Sophia,32,T128,F6
789,Local,50,Oliver,45,T129,G7


In [15]:
train_df = spark.read \
.format("csv") \
.option("inferSchema",True)\
.option("header",True)\
.load("/public/trendytech/datasets/train.csv")

In [16]:
train_df.show()

+------------+----------+---------------+--------------+---+-------------+-----------+
|train_number|train_name|seats_available|passenger_name|age|ticket_number|seat_number|
+------------+----------+---------------+--------------+---+-------------+-----------+
|         123|   Express|            100|          John| 25|         T123|         A1|
|         123|   Express|            100|          Emma| 30|         T124|         B2|
|         456| Superfast|            150|       Michael| 35|         T125|         C3|
|         456| Superfast|            150|        Sophia| 40|         T126|         D4|
|         789|     Local|             50|       William| 28|         T127|         E5|
|         789|     Local|             50|        Sophia| 32|         T128|         F6|
|         789|     Local|             50|        Oliver| 45|         T129|         G7|
+------------+----------+---------------+--------------+---+-------------+-----------+



In [17]:
# 3 a) Drop the columns passenger_name and age from the dataset.
tarin_col_drop = train_df.drop("passenger_name" ,"age")
tarin_col_drop.show()

+------------+----------+---------------+-------------+-----------+
|train_number|train_name|seats_available|ticket_number|seat_number|
+------------+----------+---------------+-------------+-----------+
|         123|   Express|            100|         T123|         A1|
|         123|   Express|            100|         T124|         B2|
|         456| Superfast|            150|         T125|         C3|
|         456| Superfast|            150|         T126|         D4|
|         789|     Local|             50|         T127|         E5|
|         789|     Local|             50|         T128|         F6|
|         789|     Local|             50|         T129|         G7|
+------------+----------+---------------+-------------+-----------+



In [18]:
# 3 b) Count the number of rows after removing duplicates of columns train_number and ticket_number
train_dups = train_df.dropDuplicates(['train_number','ticket_number'])
train_dups.count()

7

In [19]:
# 3 c) Count the number of unique train names
train_df.select('train_name').distinct().count()

3

In [20]:
# 4. Using PySpark, load the data into a Dataframe and perform the following operations on the hospital dataset (/public/trendytech/datasets/hospital.csv):

In [21]:
! hadoop fs -head /public/trendytech/datasets/hospital.csv

patient_id,admission_date,discharge_date,diagnosis,doctor_id,total_cost
1,01-01-2022,2022-01-10,Pneumonia,101,5000.00
2,02-05-2022,2022-02-09,Appendicitis,102,7000.00
3,03-12-2022,2022-03-18,Fractured Arm,103,3500.00
4,04-02-2022,2022-04-08,Heart Attack,104,15000.00
5,05-05-2022,2022-05-07,Influenza,105,2500.00
6,06-10-2022,2022-06-15,Appendicitis,106,8000.00
7,07-20-2022,2022-07-25,Pneumonia,107,5500.00
8,08-25-2022,2022-09-01,Heart Attack,108,20000.00
9,09-15-2022,2022-09-22,Fractured Leg,109,6000.00
10,10-05-2022,2022-10-10,Appendicitis,110,7500.00
11,11-02-2022,2022-11-05,Influenza,111,2800.00
12,12-10-2022,2022-12-18,Pneumonia,112,6000.00
13,01-02-2023,2023-01-09,Heart Attack,113,18000.00
14,02-14-2023,2023-02-18,Appendicitis,114,7200.00
15,03-20-2023,2023-03-28,Fractured Arm,115,3800.00
16,04-05-2023,2023-04-11,Influenza,116,2700.00
17,05-08-2023,2023-05-11,Heart Attack,117,16000.00
18,06-15-2023,2023-06-20,Pneumonia,118,4800.00
19,07-22-2023,2023-07-27,Fractured Leg,119,6500.00


In [22]:
hospital_df =  spark.read \
.format('csv') \
.option("header",True) \
.option("inferSchema",True) \
.load("/public/trendytech/datasets/hospital.csv")

In [23]:
hospital_df.printSchema()
# Date fields are loaded as null , will use to_date and convert to date in the next steps
# or while loading the df we can use .option("dateFormat",'yyy-mm-dd')

root
 |-- patient_id: integer (nullable = true)
 |-- admission_date: string (nullable = true)
 |-- discharge_date: string (nullable = true)
 |-- diagnosis: string (nullable = true)
 |-- doctor_id: integer (nullable = true)
 |-- total_cost: double (nullable = true)



In [24]:
# 4 a) Drop the "doctor_id" column from the dataset.
hospital_df = hospital_df.drop('doctor_id')
hospital_df.show()

+----------+--------------+--------------+-------------+----------+
|patient_id|admission_date|discharge_date|    diagnosis|total_cost|
+----------+--------------+--------------+-------------+----------+
|         1|    01-01-2022|    2022-01-10|    Pneumonia|    5000.0|
|         2|    02-05-2022|    2022-02-09| Appendicitis|    7000.0|
|         3|    03-12-2022|    2022-03-18|Fractured Arm|    3500.0|
|         4|    04-02-2022|    2022-04-08| Heart Attack|   15000.0|
|         5|    05-05-2022|    2022-05-07|    Influenza|    2500.0|
|         6|    06-10-2022|    2022-06-15| Appendicitis|    8000.0|
|         7|    07-20-2022|    2022-07-25|    Pneumonia|    5500.0|
|         8|    08-25-2022|    2022-09-01| Heart Attack|   20000.0|
|         9|    09-15-2022|    2022-09-22|Fractured Leg|    6000.0|
|        10|    10-05-2022|    2022-10-10| Appendicitis|    7500.0|
|        11|    11-02-2022|    2022-11-05|    Influenza|    2800.0|
|        12|    12-10-2022|    2022-12-18|    Pn

In [25]:
# 4 .b) Rename the "total_cost" column to "hospital_bill"
hospital_df  = hospital_df.withColumnRenamed('total_cost','hospital_bill')
hospital_df.show()

+----------+--------------+--------------+-------------+-------------+
|patient_id|admission_date|discharge_date|    diagnosis|hospital_bill|
+----------+--------------+--------------+-------------+-------------+
|         1|    01-01-2022|    2022-01-10|    Pneumonia|       5000.0|
|         2|    02-05-2022|    2022-02-09| Appendicitis|       7000.0|
|         3|    03-12-2022|    2022-03-18|Fractured Arm|       3500.0|
|         4|    04-02-2022|    2022-04-08| Heart Attack|      15000.0|
|         5|    05-05-2022|    2022-05-07|    Influenza|       2500.0|
|         6|    06-10-2022|    2022-06-15| Appendicitis|       8000.0|
|         7|    07-20-2022|    2022-07-25|    Pneumonia|       5500.0|
|         8|    08-25-2022|    2022-09-01| Heart Attack|      20000.0|
|         9|    09-15-2022|    2022-09-22|Fractured Leg|       6000.0|
|        10|    10-05-2022|    2022-10-10| Appendicitis|       7500.0|
|        11|    11-02-2022|    2022-11-05|    Influenza|       2800.0|
|     

In [26]:
# 4 c). Add a new column called "duration_of_stay" that represents the number of days a patient stayed in the hospital. 
    # (hint: The duration should be calculated as the difference between the "discharge_date" and "admission_date" columns)

In [27]:
from pyspark.sql.functions import to_date, expr

In [28]:
hospital_df = hospital_df.withColumn('admission_date',to_date('admission_date','dd-mm-yyyy'))
hospital_df = hospital_df.withColumn('discharge_date',to_date('discharge_date','yyyy-mm-dd'))

In [29]:
hospital_df.printSchema()
# admission_date & discharge_date are convered to date

root
 |-- patient_id: integer (nullable = true)
 |-- admission_date: date (nullable = true)
 |-- discharge_date: date (nullable = true)
 |-- diagnosis: string (nullable = true)
 |-- hospital_bill: double (nullable = true)



In [30]:
hospital_df = hospital_df.withColumn('duration_of_stay',expr("dateDiff(discharge_date,admission_date)"))
hospital_df.show()

+----------+--------------+--------------+-------------+-------------+----------------+
|patient_id|admission_date|discharge_date|    diagnosis|hospital_bill|duration_of_stay|
+----------+--------------+--------------+-------------+-------------+----------------+
|         1|    2022-01-01|    2022-01-10|    Pneumonia|       5000.0|               9|
|         2|    2022-01-02|    2022-01-09| Appendicitis|       7000.0|               7|
|         3|    2022-01-03|    2022-01-18|Fractured Arm|       3500.0|              15|
|         4|    2022-01-04|    2022-01-08| Heart Attack|      15000.0|               4|
|         5|    2022-01-05|    2022-01-07|    Influenza|       2500.0|               2|
|         6|    2022-01-06|    2022-01-15| Appendicitis|       8000.0|               9|
|         7|    2022-01-07|    2022-01-25|    Pneumonia|       5500.0|              18|
|         8|    2022-01-08|    2022-01-01| Heart Attack|      20000.0|              -7|
|         9|    2022-01-09|    2

In [31]:
#  4 d) Create a new column called "adjusted_total_cost" that calculates the adjusted total cost based on the diagnosis as follows:
        #  If the diagnosis is "Heart Attack", multiply the hospital_bill by 1.5.
        #  If the diagnosis is "Appendicitis", multiply the hospital_bill by 1.2.
        #  For any other diagnosis, keep the hospital_bill as it is.

In [32]:
Hospital_prefinal = hospital_df.withColumn('adjusted_total_cost',expr("CASE WHEN diagnosis LIKE '%Heart Attack%' THEN  hospital_bill * 1.5 WHEN diagnosis LIKE '%Appendicitis%' THEN  hospital_bill * 1.2 ELSE hospital_bill END"))
Hospital_prefinal.show()

+----------+--------------+--------------+-------------+-------------+----------------+-------------------+
|patient_id|admission_date|discharge_date|    diagnosis|hospital_bill|duration_of_stay|adjusted_total_cost|
+----------+--------------+--------------+-------------+-------------+----------------+-------------------+
|         1|    2022-01-01|    2022-01-10|    Pneumonia|       5000.0|               9|             5000.0|
|         2|    2022-01-02|    2022-01-09| Appendicitis|       7000.0|               7|             8400.0|
|         3|    2022-01-03|    2022-01-18|Fractured Arm|       3500.0|              15|             3500.0|
|         4|    2022-01-04|    2022-01-08| Heart Attack|      15000.0|               4|            22500.0|
|         5|    2022-01-05|    2022-01-07|    Influenza|       2500.0|               2|             2500.0|
|         6|    2022-01-06|    2022-01-15| Appendicitis|       8000.0|               9|             9600.0|
|         7|    2022-01-07| 

In [33]:
Hospital_prefinal.show()

+----------+--------------+--------------+-------------+-------------+----------------+-------------------+
|patient_id|admission_date|discharge_date|    diagnosis|hospital_bill|duration_of_stay|adjusted_total_cost|
+----------+--------------+--------------+-------------+-------------+----------------+-------------------+
|         1|    2022-01-01|    2022-01-10|    Pneumonia|       5000.0|               9|             5000.0|
|         2|    2022-01-02|    2022-01-09| Appendicitis|       7000.0|               7|             8400.0|
|         3|    2022-01-03|    2022-01-18|Fractured Arm|       3500.0|              15|             3500.0|
|         4|    2022-01-04|    2022-01-08| Heart Attack|      15000.0|               4|            22500.0|
|         5|    2022-01-05|    2022-01-07|    Influenza|       2500.0|               2|             2500.0|
|         6|    2022-01-06|    2022-01-15| Appendicitis|       8000.0|               9|             9600.0|
|         7|    2022-01-07| 

In [34]:
# 4 e)Select the "patient_id", "diagnosis", "hospital_bill", and  "adjusted_total_cost" columns.

In [35]:
Hospital_final_df = Hospital_prefinal.select('patient_id','diagnosis','hospital_bill','adjusted_total_cost')
Hospital_final_df.show()

+----------+-------------+-------------+-------------------+
|patient_id|    diagnosis|hospital_bill|adjusted_total_cost|
+----------+-------------+-------------+-------------------+
|         1|    Pneumonia|       5000.0|             5000.0|
|         2| Appendicitis|       7000.0|             8400.0|
|         3|Fractured Arm|       3500.0|             3500.0|
|         4| Heart Attack|      15000.0|            22500.0|
|         5|    Influenza|       2500.0|             2500.0|
|         6| Appendicitis|       8000.0|             9600.0|
|         7|    Pneumonia|       5500.0|             5500.0|
|         8| Heart Attack|      20000.0|            30000.0|
|         9|Fractured Leg|       6000.0|             6000.0|
|        10| Appendicitis|       7500.0|             9000.0|
|        11|    Influenza|       2800.0|             2800.0|
|        12|    Pneumonia|       6000.0|             6000.0|
|        13| Heart Attack|      18000.0|            27000.0|
|        14| Appendiciti

In [36]:
# spark.stop()